In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import numpy as np
import torch.optim as optim

In [30]:
class CustomDataset(Dataset):
    def __init__(self, filepath, word2idx, tag2idx):
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.sentences, self.ner_tags = self.load_data(filepath)

    def load_data(self, filepath):
        sentences, ner_tags = [], []
        sentence, ner_tag = [], []
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    parts = line.split("\t")
                    if len(parts) >= 3:
                        word, _, ner = parts
                        sentence.append(word)
                        ner_tag.append(ner)
                else:
                    if sentence:
                        sentences.append(sentence)
                        ner_tags.append(ner_tag)
                        sentence, ner_tag = [], []
            # In case file does not end with a newline
            if sentence:
                sentences.append(sentence)
                ner_tags.append(ner_tag)
        return sentences, ner_tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        word_ids = [self.word2idx.get(w, self.word2idx["<UNK>"]) for w in self.sentences[idx]]
        tag_ids = [self.tag2idx.get(t, self.tag2idx["<UNK>"]) for t in self.ner_tags[idx]]
        return word_ids, tag_ids

def collate_fn(batch):
    sentences, tags = zip(*batch)
    max_len = max(len(s) for s in sentences)

    pad_token = 0  # word2idx["<PAD>"]
    padded_sentences = [s + [pad_token] * (max_len - len(s)) for s in sentences]
    padded_tags = [t + [pad_token] * (max_len - len(t)) for t in tags]

    return torch.tensor(padded_sentences, dtype=torch.long), torch.tensor(padded_tags, dtype=torch.long)


In [31]:
def extract_unique_tags(filepath):
    tag_set = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split("\t")
                if len(parts) >= 3:
                    tag = parts[2]
                    tag_set.add(tag)
    return sorted(tag_set)


Load Dataset

In [32]:
train_path = "/content/drive/MyDrive/Datasets/train_v5.conll"
val_path = "/content/drive/MyDrive/Datasets/val_v5.conll"
test_path = "/content/drive/MyDrive/Datasets/test_v5.conll"

word2idx = {"<PAD>": 0, "<UNK>": 1}
tag2idx = {"<PAD>": 0, "<UNK>": 1}
train_data = CustomDataset(train_path,word2idx, tag2idx)
val_data = CustomDataset(val_path,word2idx, tag2idx)
test_data = CustomDataset(test_path,word2idx, tag2idx)

train_load = DataLoader(train_data, batch_size=32, collate_fn=collate_fn)
val_load = DataLoader(val_data, batch_size=32, collate_fn=collate_fn)


In [33]:
unique_tags = extract_unique_tags(train_path)
tag2idx = {tag: idx for idx, tag in enumerate(unique_tags)}
tag2idx["<PAD>"] = len(tag2idx)  # Add PAD if needed
tag2idx["<UNK>"] = len(tag2idx)  # Optional: Add UNK tag

print("Number of tags:", len(tag2idx))
print(tag2idx)


Number of tags: 27
{'B-DATE': 0, 'B-LOC': 1, 'B-NUM': 2, 'B-ORG': 3, 'B-PER': 4, 'B-TIME': 5, 'E-DATE': 6, 'E-LOC': 7, 'E-NUM': 8, 'E-ORG': 9, 'E-PER': 10, 'E-TIME': 11, 'I-DATE': 12, 'I-LOC': 13, 'I-NUM': 14, 'I-ORG': 15, 'I-PER': 16, 'I-TIME': 17, 'O': 18, 'S-DATE': 19, 'S-LOC': 20, 'S-NUM': 21, 'S-ORG': 22, 'S-PER': 23, 'S-TIME': 24, '<PAD>': 25, '<UNK>': 26}


In [34]:
for batch in train_load:
    sentences, tags = batch
    print("Batch max len:", sentences.shape)
    print("tags:", tags.shape)
    break


Batch max len: torch.Size([32, 41])
tags: torch.Size([32, 41])


In [35]:
# #Force reinstall compatible versions
# !pip install gensim
# !pip install numpy==1.24.3 --force-reinstall

In [36]:
from gensim.models import KeyedVectors
fasttext_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Datasets/cc.my.300.vec', binary=False)
# https://fasttext.cc/docs/en/crawl-vectors.html choose Burmese choose text .vec file

In [37]:
def build_vocab(dataset):
    word_set = set()
    for sentence in dataset.sentences:
        for word in sentence:
            word_set.add(word)
    return word_set

vocab = build_vocab(train_data)
vocab_size = len(vocab)
embedding_dim = 300
embedding_matrix = []

# Add PAD and UNK embeddings
embedding_matrix.append(np.zeros(embedding_dim))  # <PAD>
embedding_matrix.append(np.random.uniform(-0.25, 0.25, embedding_dim))  # <UNK>


for word in vocab:
    word2idx[word] = len(word2idx)
    if word in fasttext_model:
        embedding_matrix.append(fasttext_model[word])
    else:
        embedding_matrix.append(np.random.uniform(-0.25, 0.25, embedding_dim))

embedding_matrix = torch.tensor(embedding_matrix).float()

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    """
    Feedforward neural network with dropout and GELU activation.

    Parameters:
    - dim: Input dimension
    - expension_factor: Factor by which the hidden layer dimension will expand
    - dropout: Dropout probability for regularization
    """
    def __init__(self, dim, expension_factor, dropout):
        super(FeedForward, self).__init__()
        hidden_dim = dim * expension_factor
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, dim)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout1(F.gelu(self.fc1(x)))  # Apply GELU and dropout on the first layer
        return self.dropout2(self.fc2(x))  # Apply second linear layer with dropout

class Fourier(nn.Module):
    """
    Fourier layer applying FFT twice (along two dimensions).

    Parameters:
    - dropout: Dropout probability
    """
    def __init__(self, dropout=0.3):
        super(Fourier, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.act = nn.ReLU()

    def forward(self, x):
        # Apply FFT along the last dimension (real and imaginary parts)
        x = x.to(dtype=torch.float64)
        x = torch.fft.fft(x, dim=-1)
        x = torch.fft.fft(x, dim=1)  # Apply FFT along the second dimension as well
        x = self.act(x.real)  # Using only real part
        x = x.to(dtype=torch.float32)
        x = self.dropout(x)
        return x

class FNet(nn.Module):
    """
    FNet model combining Fourier transformations and FeedForward layers.

    Parameters:
    - dim: Input dimension
    - expension_factor: Expansion factor for FeedForward layers
    - dropout: Dropout probability for regularization
    """
    def __init__(self, dim, expension_factor, dropout,embedding_matrix):

        super(FNet, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)  # Freeze embeddings
        self.fourier = Fourier(dropout)
        self.ffn = FeedForward(dim, expension_factor, dropout)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)

    def forward(self, x):
        x = self.embedding(x)
        residual = x
        x = self.fourier(x)
        x = self.norm1(x + residual)
        residual = x
        x = self.ffn(x)
        out = self.norm2(x + residual)
        return out

# Final NER model with classification head
class FNetNER(nn.Module):
    def __init__(self, embedding_matrix, num_tags, expension_factor=4, dropout=0.3):
        super(FNetNER, self).__init__()
        dim = embedding_matrix.shape[1]
        self.fnet = FNet(dim, expension_factor, dropout, embedding_matrix)
        self.classifier = nn.Linear(dim, num_tags)

    def forward(self, x):
        x = self.fnet(x)  # [batch_size, seq_len, dim]
        logits = self.classifier(x)  # [batch_size, seq_len, num_tags]
        return logits


In [39]:
print(embedding_matrix.shape)
print(len(word2idx))
print((len(tag2idx)))

torch.Size([16736, 300])
16736
27


In [40]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print("Number of tags:", len(tag2idx))


Number of tags: 27


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FNetNER(embedding_matrix, num_tags=len(tag2idx), expension_factor=4, dropout=0.3).to(device)
optimizer = optim.Adam(model.parameters(), lr =3e-4)
criterion = nn.CrossEntropyLoss(ignore_index = tag2idx["<PAD>"])

In [42]:
# Training loop
num_epochs = 25

for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for sentences, tags in train_load:
        sentences, tags = sentences.to(device), tags.to(device)

        optimizer.zero_grad()
        outputs = model(sentences)

        outputs = outputs.view(-1, outputs.shape[-1])  # [batch_size * seq_len, num_tags]
        tags = tags.view(-1)                           # [batch_size * seq_len]

        loss = criterion(outputs, tags)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_load)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for sentences, tags in val_load:
            sentences, tags = sentences.to(device), tags.to(device)
            outputs = model(sentences)

            outputs = outputs.view(-1, outputs.shape[-1])
            tags = tags.view(-1)
            loss = criterion(outputs, tags)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_load)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")


Epoch 1/2 - Train Loss: 0.4597 - Val Loss: 0.3081
Epoch 2/2 - Train Loss: 0.2740 - Val Loss: 0.2634


In [43]:
#Test case
test_sentence = ["ရန်ကုန်", "မြို့", "သို့", "သွားသည်။"]
word_ids = [word2idx.get(word, word2idx["<UNK>"]) for word in test_sentence]
input_tensor = torch.tensor([word_ids], dtype=torch.long).to(device)  # Shape: [1, seq_len]
model.eval()
with torch.no_grad():
    output = model(input_tensor)  # [1, seq_len, num_tags]
    predicted_ids = torch.argmax(output, dim=-1).squeeze(0).tolist()  # Remove batch dim
# Invert the tag2idx dictionary
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
predicted_tags = [idx2tag[idx] for idx in predicted_ids]
for word, tag in zip(test_sentence, predicted_tags):
    print(f"{word}\t{tag}")


ရန်ကုန်	B-DATE
မြို့	B-LOC
သို့	B-LOC
သွားသည်။	B-DATE
